<a href="https://colab.research.google.com/github/SunSlick2/booktrade/blob/main/show_the_code_againinclude_the_followingimport_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Got it. I'll update the Python script to use `(top_left_x, top_left_y)` and `(bottom_right_x, bottom_right_y)` coordinates, and calculate width/height from those. I'll also include the `KMP_DUPLICATE_LIB_OK` environment variable setting and ensure the `model_storage_directory` is explicitly set to the user's EasyOCR model path.

Here's the revised Python code:

In [ ]:
import mss
import mss.tools
from PIL import Image
import easyocr
import re
import os
import sys

# --- Set OpenMP environment variable to suppress warnings (must be at the very top) ---
# This helps prevent "OMP: Error #15" warnings related to multiple OpenMP runtimes.
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# --- Configuration ---
# Define the snip coordinates as (top_left_x, top_left_y) and (bottom_right_x, bottom_right_y)
top_left_x = 317
top_left_y = 419
bottom_right_x = 428
bottom_right_y = 456

# Calculate width and height from the provided coordinates
capture_region = {
    "left": top_left_x,
    "top": top_left_y,
    "width": bottom_right_x - top_left_x,
    "height": bottom_right_y - top_left_y
}

# Define the path for the output file
output_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), "extracted_date_easyocr.txt")

# Define the directory where EasyOCR models are stored locally (e.g., after manual download)
# This will typically be C:\Users\YourUser\.EasyOCR\model on Windows
# os.path.expanduser('~') resolves to the user's home directory.
easyocr_model_dir = os.path.join(os.path.expanduser('~'), '.EasyOCR', 'model')

# Initialize EasyOCR reader once globally for efficiency
# This uses the specified local model directory and prevents internet downloads.
try:
    reader = easyocr.Reader(
        ['en'], # Languages to use for OCR. 'en' for English.
        model_storage_directory=easyocr_model_dir,
        download_enabled=False # Crucial: tells EasyOCR NOT to try downloading from the internet
    )
    sys.stdout.write("✅ EasyOCR reader initialized successfully from local models.\n")
except Exception as e:
    sys.stderr.write(f"❌ Error initializing EasyOCR reader. Please ensure models are in '{easyocr_model_dir}' and are correct: {e}\n")
    sys.exit(1) # Exit if reader cannot be initialized


def capture_screen_region(region):
    """Captures a specific region of the screen."""
    try:
        sys.stdout.write("Capturing screen region...\n")
        with mss.mss() as sct:
            sct_img = sct.grab(region)
            # Convert to PIL Image for processing
            img = Image.frombytes("RGB", sct_img.size, sct_img.rgb)
            return img
    except Exception as e:
        sys.stderr.write(f"❌ Error capturing screen: {e}\n")
        return None

def ocr_image_for_date(image):
    """Performs OCR on an image and tries to find a date using EasyOCR."""
    if image is None:
        return None

    try:
        sys.stdout.write("Performing OCR with EasyOCR...\n")
        # EasyOCR returns a list of (bbox, text, confidence)
        results = reader.readtext(image)

        full_ocr_text = ""
        # Collect all text to search for the date, or directly search in each result
        for (bbox, text, prob) in results:
            full_ocr_text += text + " " # Concatenate recognized text

        sys.stdout.write(f"DEBUG: EasyOCR Raw Text (concatenated): {full_ocr_text.strip()}\n")

        # Regular expression for dd/mm/yyyy format
        # This pattern checks for valid day (01-31), month (01-12), and year (1900-2099)
        date_pattern = r'\b(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/((?:19|20)\d{2})\b'

        # Prioritize searching in individual OCR results for better precision
        for (bbox, text, prob) in results:
            match = re.search(date_pattern, text)
            if match:
                return match.group(0) # Return the first found date

        # If no date found in individual results, try the full concatenated text
        match = re.search(date_pattern, full_ocr_text)
        if match:
            return match.group(0)

        return None # No date found

    except Exception as e:
        sys.stderr.write(f"❌ Error during EasyOCR processing or date parsing: {e}\n")
        return None

def write_result_to_file(value, file_path):
    """Writes the extracted value to a specified file."""
    try:
        with open(file_path, "w") as f:
            f.write(value if value is not None else "") # Write empty string if no date found
    except Exception as e:
        sys.stderr.write(f"❌ Error writing to file {file_path}: {e}\n")

def main():
    extracted_date = None
    try:
        # Capture the screen region
        screenshot = capture_screen_region(capture_region)
        if screenshot:
            # Optional: Save the captured image for debugging
            screenshot_debug_path = "captured_date_region_debug.png"
            screenshot.save(screenshot_debug_path)
            sys.stdout.write(f"Captured region saved to {screenshot_debug_path}\n")

            # Perform OCR and extract the date
            extracted_date = ocr_image_for_date(screenshot)

    except Exception as e:
        sys.stderr.write(f"❌ An unexpected error occurred in main: {e}\n")
    finally:
        # Ensure the output file is always created, even if no date is found or an error occurs
        write_result_to_file(extracted_date, output_file_path)
        if extracted_date:
            sys.stdout.write(f"✔️ Successfully wrote '{extracted_date}' to {output_file_path}\n")
        else:
            sys.stdout.write(f"⚠️ No date found or error occurred. Wrote empty string to {output_file_path}\n")

if __name__ == "__main__":
    main()